Processus d'Indexation (Base Index)
Le script Generation_Index.ipynb parcourt l'ensemble des données (JSON et PDF) pour construire une base d'index inversé (Base_Index.json) et un fichier de longueurs de documents (document_lengths.json), essentiels pour le calcul TF-IDF.

Étapes Clés :
Chargement du Stock Protégé :

Le fichier protected_terms.json est chargé. Ces termes (noms scientifiques, Darija, etc.) sont mis en minuscules et utilisés pour protéger des termes spécifiques de la lemmatisation.

Parcours et Extraction des Documents :

Le code itère sur tous les fichiers JSON (../docs/Plantes) et PDF (../docs/Concepts).

Les ID des documents sont basés sur le nom de fichier.

Traitement des Termes Spéciaux (Indexation Directe) :

Nombres : Les valeurs numériques (ex: 30, 1.5) sont extraites du texte et ajoutées aux tokens sans modification.

Darija : Les termes du champ noms_darija sont extraits et ajoutés tels quels, en minuscules.

Reconnaissance des Termes Scientifiques (N-grams) :

Le texte général est tokenisé.

Les séquences de tokens (jusqu'à 4 mots, ou N-grams) sont comparées au Stock Protégé.

Si une séquence correspond (ex: "petroselinum crispum"), elle est capturée comme un terme unique et ajoutée aux tokens indexables. Les tokens composant ce terme sont exclus de la lemmatisation.

Normalisation et Lemmatisation (Mots Généraux) :

Les tokens restants (mots généraux) sont traités par spaCy.

Ils sont filtrés (suppression des stop words, ponctuation, symboles).

Ils sont réduits à leur lemme (ex: "cultivées" → "cultivé").

Construction de l'Index Inversé (TF) :

L'index est rempli en comptant la fréquence brute (TF) de chaque occurrence de chaque token (lemme ou terme spécial) dans chaque document.

Calcul des Longueurs :

La longueur du document est définie comme le nombre total de tokens indexables (y compris les répétitions) et est enregistrée dans document_lengths.json.

Sauvegarde :

Les fichiers Base_Index.json et document_lengths.json sont sauvegardés dans le dossier de l'indexeur.

In [1]:
import os
import json
import re
from collections import defaultdict
import math
import pypdf
import spacy

# --- 1. Paramètres de Configuration et Préparation spaCy ---

# Chemins relatifs à Plant_search/plant-search-engine/indexer/
DOSSIER_PLANTES_JSON = "../docs/Plantes"
DOSSIER_CONCEPTS_PDF = "../docs/Concepts"

# FICHIER D'ENTRÉE POUR LE STOCK SCIENTIFIQUE
FICHIER_STOCK_SCIENTIFIQUE = "../docs/mot_scientifique/protected_terms.json" 

# Fichiers de sortie (dans le dossier indexer/)
FICHIER_INDEX_SORTIE = "Base_Index.json"
FICHIER_LONGUEURS_SORTIE = "document_lengths.json"

LANGUE = 'fr_core_news_sm'
MAX_NGRAM_SCIENTIFIQUE = 4 # Longueur maximale des N-grammes à vérifier

# Chargement du modèle spaCy
try:
    nlp = spacy.load(LANGUE)
except OSError:
    print(f"\n🚨 Erreur: Modèle spaCy '{LANGUE}' non trouvé.")
    print(f"Veuillez l'installer avec : python -m spacy download {LANGUE}")
    exit()

# Stock des termes scientifiques chargé au démarrage
TERMES_SCIENTIFIQUES_STOCK = set()

def charger_stock_scientifique(filepath: str) -> set:
    """
    Charge le stock scientifique depuis un fichier JSON. 
    Les termes sont convertis en minuscules pour la comparaison.
    """
    stock = set()
    if not os.path.exists(filepath):
        print(f"⚠️ Avertissement: Fichier de stock scientifique '{filepath}' non trouvé. Le stock est vide.")
        return stock
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            if isinstance(data, list):
                stock.update({item.lower().strip() for item in data})
            elif isinstance(data, dict):
                stock.update({key.lower().strip() for key in data.keys()})
            else:
                print(f"⚠️ Avertissement: Format du fichier '{filepath}' non reconnu.")

    except Exception as e:
        print(f"❌ Erreur lors du chargement du stock scientifique: {e}")
        return set()
    
    print(f"✅ Stock de {len(stock)} termes scientifiques/spéciaux chargés.")
    return stock

# Charger le stock dès que possible
TERMES_SCIENTIFIQUES_STOCK = charger_stock_scientifique(FICHIER_STOCK_SCIENTIFIQUE)


# --- 2. Fonctions d'Extraction et de Traitement du Texte ---

def extraire_texte_json_avec_termes_separes(filepath: str) -> tuple[str, list[str], list[str]]:
    """
    Extrait le texte général, les termes numériques, et les termes Darija.
    Les termes Darija sont extraits directement (tels quels, en minuscule).
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Erreur d'extraction/lecture JSON à {filepath}: {e}")
        return "", [], []
        
    texte_general = []
    termes_nombres = []
    termes_darija_directs = [] # Pour les termes Darija (pris tels quels)
    
    def parcourir_dict(d):
        for key, value in d.items():
            if key in ('url', 'urls', 'galerie_images', 'id'):
                continue
                
            # GESTION SPÉCIALE DES NOMS DARIJA (Indexation Directe)
            if key == 'noms_darija' and isinstance(value, list):
                # Ajout des termes Darija en minuscules, tels quels, sans lemmatisation
                termes_darija_directs.extend([t.lower().strip() for t in value if t])
                continue # NE PAS AJOUTER AU TEXTE GÉNÉRAL

            # Le reste (y compris 'nom_scientifique') est ajouté au texte général
            if isinstance(value, dict):
                parcourir_dict(value)
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, dict):
                        parcourir_dict(item)
                    elif isinstance(item, str):
                        texte_general.append(item)
            elif isinstance(value, str):
                texte_general.append(value)
    
    parcourir_dict(data)
    
    texte_complet = ' '.join(texte_general)
    
    # 3. Extraction des Nombres (sans leurs unités)
    nombres = re.findall(r'(\d+[\.,]\d+|\d+)\s*(%|ml|g|cm|mm|m|l)?', texte_complet, re.IGNORECASE)
    
    for nombre, unite in nombres:
        terme_nombre = nombre.replace(',', '.') 
        termes_nombres.append(terme_nombre)
    
    # Nettoyer le texte général des chiffres pour le traitement NLP
    texte_nettoye = re.sub(r'(\d+[\.,]\d+|\d+)\s*(%|ml|g|cm|mm|m|l)?', ' ', texte_complet, flags=re.IGNORECASE)
        
    return texte_nettoye, termes_nombres, termes_darija_directs

def extraire_texte_pdf(filepath: str) -> str:
    """Extrait le texte d'un fichier PDF."""
    texte = ""
    try:
        with open(filepath, 'rb') as f:
            reader = pypdf.PdfReader(f)
            for page in reader.pages:
                texte += page.extract_text() + "\n"
    except Exception as e:
        print(f"⚠️ Erreur lors de l'extraction de texte du PDF {filepath}. Erreur: {e}")
        return ""
    return texte

def identifier_termes_scientifiques(tokens: list, stock_scientifique: set, max_n: int) -> tuple[list, list]:
    """
    Identifie et extrait les termes scientifiques/spéciaux multi-mots (N-grammes)
    en comparant le texte tokenisé au stock.
    """
    mots_scientifiques = []
    indices_captures = set() 
    
    i = 0
    while i < len(tokens):
        if i in indices_captures:
            i += 1
            continue
        
        terme_trouve = False
        
        # Teste les N-grammes de la plus grande taille à la plus petite (ex: 4, 3, 2, 1)
        for n in range(min(max_n, len(tokens) - i), 0, -1):
            
            ngram_tokens = tokens[i:i+n]
            
            # Concaténation des tokens pour la comparaison
            ngram_candidat = " ".join(ngram_tokens)
            ngram_candidat_lower = ngram_candidat.lower().strip()
            
            if ngram_candidat_lower in stock_scientifique:
                # Terme scientifique/spécial trouvé ! On l'ajoute tel quel.
                mots_scientifiques.append(ngram_candidat_lower)
                
                # Marquer tous les tokens de ce N-gramme comme capturés
                for j in range(n):
                    indices_captures.add(i + j)
                
                terme_trouve = True
                i += n 
                break
        
        if not terme_trouve:
            i += 1
            
    # Extraction des tokens restants (mots généraux)
    tokens_restants = [tokens[i] for i in range(len(tokens)) if i not in indices_captures]
    
    return mots_scientifiques, tokens_restants


def normaliser_texte_lemmatisation_filtree(tokens_restants: list) -> list[str]:
    """
    Lemmatisation stricte des tokens restants (mots généraux). 
    Applique le filtrage : minuscules, enlève ponctuation, stop words, symboles, puis lemmatise.
    """
    texte_restant = " ".join(tokens_restants) 
    doc = nlp(texte_restant.lower())
    stop_words = nlp.Defaults.stop_words
    mots_normalises = []
    
    for token in doc:
        lemma_text = token.lemma_.lower().strip()
        
        # Application du filtrage strict pour les mots généraux
        if (
            not token.is_punct and      # Enlève la ponctuation
            token.is_alpha and          # Enlève les symboles et caractères non alphabétiques
            not token.is_stop and       # Enlève les stop words
            len(lemma_text) > 1         # Longueur minimale du lemme
        ):
            mots_normalises.append(lemma_text) 
            
    return mots_normalises

# --- 3. Classe Principale d'Indexation ---

class IndexeurBotanique:
    
    def __init__(self, json_dir, pdf_dir):
        self.json_dir = json_dir
        self.pdf_dir = pdf_dir
        # Structure TF simple: { 'mot': { 'document_id': fréquence } }
        self.index = defaultdict(lambda: defaultdict(int))
        self.doc_lengths = {}
        
    def _creer_doc_id(self, filename: str, doc_type: str) -> str:
        """Crée l'ID du document en utilisant LE NOM DE FICHIER COMPLET (avec extension)."""
        return filename

    def parcourir_documents(self):
        """Parcourt et indexe TOUS les JSON et TOUS les PDF."""
        documents_a_indexer = []
        
        print(f"Recherche des fichiers JSON dans '{self.json_dir}'...")
        if os.path.isdir(self.json_dir):
            for filename in os.listdir(self.json_dir):
                if filename.endswith(".json"):
                    doc_id = self._creer_doc_id(filename, 'json')
                    documents_a_indexer.append({'id': doc_id, 'type': 'json', 'path': os.path.join(self.json_dir, filename)})
        else:
            print(f"❌ Erreur: Dossier JSON '{self.json_dir}' introuvable.")


        print(f"Recherche des fichiers PDF dans '{self.pdf_dir}'...")
        if os.path.isdir(self.pdf_dir):
            for filename in os.listdir(self.pdf_dir):
                if filename.endswith(".pdf"):
                    doc_id = self._creer_doc_id(filename, 'pdf')
                    documents_a_indexer.append({'id': doc_id, 'type': 'pdf', 'path': os.path.join(self.pdf_dir, filename)})
        else:
            print(f"❌ Erreur: Dossier PDF '{self.pdf_dir}' introuvable.")
                
        print(f"\nDébut de l'indexation de {len(documents_a_indexer)} documents...")
        
        for doc_info in documents_a_indexer:
             self.indexer_document(doc_info)
        print("\n--- Indexation Terminée ---")

    # La fonction parcourir_documents_test() est conservée mais non utilisée dans le main
    def parcourir_documents_test(self, nom_json: str, nom_pdf: str):
         """Parcourt et indexe seulement 1 JSON et 1 PDF spécifiques pour le test."""
         # ... (votre implémentation)

    def indexer_document(self, doc_info):
        """Extrait, normalise et construit l'index TF pour un document unique."""
        doc_id = doc_info['id']
        filepath = doc_info['path']
        
        mots_indexables = []
        texte_brut_pour_tokenisation = ""
        termes_nombres = []
        termes_darija_directs = []

        # 1. Extraction du texte, nombres et termes Darija
        if doc_info['type'] == 'json':
            texte_brut_pour_tokenisation, termes_nombres, termes_darija_directs = extraire_texte_json_avec_termes_separes(filepath)
            
        elif doc_info['type'] == 'pdf':
            texte_complet_pdf = extraire_texte_pdf(filepath)
            if not texte_complet_pdf: return 
            
            # Extraction des Nombres (PDF)
            nombres = re.findall(r'(\d+[\.,]\d+|\d+)\s*(%|ml|g|cm|mm|m|l)?', texte_complet_pdf, re.IGNORECASE)
            for nombre, unite in nombres:
                 termes_nombres.append(nombre.replace(',', '.')) 
            
            # Nettoyage du texte général des chiffres pour le traitement NLP
            texte_brut_pour_tokenisation = re.sub(r'(\d+[\.,]\d+|\d+)\s*(%|ml|g|cm|mm|m|l)?', ' ', texte_complet_pdf, flags=re.IGNORECASE)

        # 2. AJOUT DIRECT des Termes (Nombres et Darija)
        mots_indexables.extend(termes_nombres)
        mots_indexables.extend(termes_darija_directs) 
        
        # 3. Tokenisation pour la reconnaissance des N-grammes
        doc = nlp(texte_brut_pour_tokenisation)
        tokens_bruts = [token.text for token in doc if not token.is_space]
        
        # 4. Reconnaissance des Termes Scientifiques (non lemmatisés)
        mots_scientifiques, tokens_restants = identifier_termes_scientifiques(
            tokens_bruts, TERMES_SCIENTIFIQUES_STOCK, MAX_NGRAM_SCIENTIFIQUE
        )
        mots_indexables.extend(mots_scientifiques) 
        
        # 5. Lemmatisation des Mots Généraux Restants
        mots_lemmatises = normaliser_texte_lemmatisation_filtree(tokens_restants)
        mots_indexables.extend(mots_lemmatises)

        if not mots_indexables: return

        # 6. Enregistrement de la longueur et construction de l'index
        self.doc_lengths[doc_id] = len(mots_indexables)
        
        for mot in mots_indexables:
            if mot:
                self.index[mot][doc_id] += 1 

    def enregistrer_longueurs(self, output_file):
        """Enregistre les longueurs des documents."""
        os.makedirs(os.path.dirname(output_file) or '.', exist_ok=True)
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(self.doc_lengths, f, ensure_ascii=False, indent=2)
        print(f"✅ Longueurs des documents (Token Counts) enregistrées dans '{output_file}'.")

    def enregistrer_index(self, output_file):
        """Enregistre l'index inversé final (Base d'Index) au format simplifié."""
        os.makedirs(os.path.dirname(output_file) or '.', exist_ok=True)
        
        index_a_sauvegarder = dict(self.index)
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(index_a_sauvegarder, f, ensure_ascii=False, indent=2)
            
        print(f"✅ Base d'Index finale ('{output_file}') enregistrée.")


# --- 4. Exécution du Script Principal (Mode Production) ---

if __name__ == "__main__":
    
    print("--- Démarrage de l'Indexation Complète (Méthode Avancée) ---")

    # Le stock scientifique est déjà chargé au début du script

    # 1. Initialisation de l'Indexeur
    indexeur = IndexeurBotanique(
        json_dir=DOSSIER_PLANTES_JSON,
        pdf_dir=DOSSIER_CONCEPTS_PDF
    )
    
    # 2. Exécution de l'indexation sur TOUS les fichiers
    indexeur.parcourir_documents()
    
    # 3. Enregistrement des Fichiers de Sortie
    indexeur.enregistrer_longueurs(FICHIER_LONGUEURS_SORTIE)
    indexeur.enregistrer_index(FICHIER_INDEX_SORTIE)

    print("\n--- Processus d'Indexation Complet Terminé ---")
    print(f"L'index final ('{FICHIER_INDEX_SORTIE}') et les longueurs ('{FICHIER_LONGUEURS_SORTIE}') sont générés dans le dossier de l'indexeur.")

✅ Stock de 615 termes scientifiques/spéciaux chargés.
--- Démarrage de l'Indexation Complète (Méthode Avancée) ---
Recherche des fichiers JSON dans '../docs/Plantes'...
Recherche des fichiers PDF dans '../docs/Concepts'...

Début de l'indexation de 95 documents...

--- Indexation Terminée ---
✅ Longueurs des documents (Token Counts) enregistrées dans 'document_lengths.json'.
✅ Base d'Index finale ('Base_Index.json') enregistrée.

--- Processus d'Indexation Complet Terminé ---
L'index final ('Base_Index.json') et les longueurs ('document_lengths.json') sont générés dans le dossier de l'indexeur.


Ensuite genration des vocabulaires qui contient que les tokens pour les json et pour les pdf  separés.
pour un usage ulterieur.

In [9]:
import os
import json
from typing import Set, Dict, List, Tuple

# --- 1. Chemins des Fichiers ---

FICHIER_INDEX_SORTIE = "Base_Index.json"
# Nouveaux fichiers de sortie
FICHIER_TOKEN_PLANTE = "token_plante.json"
FICHIER_TOKEN_CONCEPT = "token_concept.json"

# Définition des préfixes ou extensions pour identifier les types de documents
PREFIXE_PLANTE = ".json"  # ID de documents JSON
PREFIXE_CONCEPT = ".pdf" # ID de documents PDF

# --- 2. Fonction de Chargement ---

def charger_base_index(filepath: str) -> Dict[str, Dict[str, int]]:
    """Charge la Base d'Index Inversé."""
    if not os.path.exists(filepath):
        print(f"❌ ERREUR : Fichier Base d'Index non trouvé à {filepath}. Annulation.")
        return {}
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, dict):
                print(f"✅ Base d'Index chargée. Contient {len(data)} tokens uniques.")
                # La structure attendue est Dict[token, Dict[doc_id, freq]]
                return data
            else:
                print("❌ ERREUR : Le fichier Base d'Index n'est pas un dictionnaire.")
                return {}
    except json.JSONDecodeError:
        print("❌ ERREUR JSON : Le fichier Base d'Index n'est pas un JSON valide.")
        return {}
    except Exception as e:
        print(f"❌ ERREUR de lecture : {e}")
        return {}

# --- 3. Fonction d'Extraction et de Tri (CORRIGÉE) ---

def extraire_et_trier_tokens(base_index: Dict[str, Dict[str, int]]) -> Tuple[Set[str], Set[str]]:
    """
    Parcourt l'index (clé=token, valeur=Dict[doc_id, freq]) et trie chaque token 
    dans les ensembles Plantes et Concepts.
    """
    
    tokens_plantes: Set[str] = set()
    tokens_concepts: Set[str] = set()
    
    total_tokens_parcourus = 0
    
    # Itération sur les tokens (Clé = le mot/token)
    for token, postings in base_index.items():
        total_tokens_parcourus += 1
        
        token_nettoye = token.lower().strip()
        if not token_nettoye:
            continue
            
        est_plante = False
        est_concept = False
        
        # Itération sur le dictionnaire de postings (Clé = doc_id, Valeur = fréquence)
        for doc_id, frequence in postings.items(): 
            
            # Classification basée sur la fin du doc_id
            if doc_id.endswith(PREFIXE_PLANTE):
                est_plante = True
            elif doc_id.endswith(PREFIXE_CONCEPT):
                est_concept = True
            
            # Si nous avons trouvé les deux catégories, nous pouvons arrêter cette boucle interne
            if est_plante and est_concept:
                break 
                
        # Stockage du token dans les ensembles finaux
        if est_plante:
            tokens_plantes.add(token_nettoye)
        
        if est_concept:
            tokens_concepts.add(token_nettoye)
            
    print(f"\n--- Résultat du Tri ---")
    print(f"Total des tokens uniques dans l'Index : {total_tokens_parcourus}")
    print(f"Tokens classés comme 'Plantes' (JSON) : {len(tokens_plantes)}")
    print(f"Tokens classés comme 'Concepts' (PDF) : {len(tokens_concepts)}")
    
    return tokens_plantes, tokens_concepts

# --- 4. Fonction de Sauvegarde ---

def sauvegarder_vocabulaire(vocabulaire: Set[str], filepath: str, nom_type: str):
    """Sauvegarde l'ensemble de tokens trié dans un fichier JSON."""
    try:
        liste_triee = sorted(list(vocabulaire))
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(liste_triee, f, ensure_ascii=False, indent=2)
            
        print(f"✅ Vocabulaire {nom_type} enregistré dans '{filepath}' ({len(liste_triee)} mots).")
    except Exception as e:
        print(f"❌ ERREUR lors de la sauvegarde du fichier {filepath} : {e}")

# --- 5. Exécution du Script Principal ---

if __name__ == "__main__":
    
    print("--- Démarrage de la Reconstruction du Vocabulaire depuis l'Index ---")

    # 1. Chargement de la Base d'Index
    base_index = charger_base_index(FICHIER_INDEX_SORTIE)
    
    if not base_index:
        exit()

    # 2. Extraction et Tri des Tokens
    tokens_plantes, tokens_concepts = extraire_et_trier_tokens(base_index)
    
    # 3. Sauvegarde des Nouveaux Fichiers de Vocabulaire
    sauvegarder_vocabulaire(tokens_plantes, FICHIER_TOKEN_PLANTE, "Plantes (JSON)")
    sauvegarder_vocabulaire(tokens_concepts, FICHIER_TOKEN_CONCEPT, "Concepts (PDF)")

    print("\n--- Reconstruction du Vocabulaire basée sur l'Index Terminée ---")

--- Démarrage de la Reconstruction du Vocabulaire depuis l'Index ---
✅ Base d'Index chargée. Contient 6748 tokens uniques.

--- Résultat du Tri ---
Total des tokens uniques dans l'Index : 6748
Tokens classés comme 'Plantes' (JSON) : 4673
Tokens classés comme 'Concepts' (PDF) : 3592
✅ Vocabulaire Plantes (JSON) enregistré dans 'token_plante.json' (4673 mots).
✅ Vocabulaire Concepts (PDF) enregistré dans 'token_concept.json' (3592 mots).

--- Reconstruction du Vocabulaire basée sur l'Index Terminée ---


teste et extraction des termes scientifiques dans les concepts en pdf,

In [ ]:
import json
import os

# --- Définition des chemins de fichiers (Ajustez si nécessaire) ---
# NOTE : J'utilise ici les noms de fichiers que vous avez fournis.
TOKEN_CONCEPTS_FILE = '../doc/Token/token_concept.json'
PROTECTED_TERMS_FILE = '../docs/mot_scientifique/protected_terms.json'
OUTPUT_FILE = '../docs/Token/science_concept.json'


def detect_scientific_concepts(tokens_path, protected_terms_path, output_path):
    """
    Détecte les mots scientifiques/techniques dans la liste des tokens 
    en les comparant à une liste de termes protégés et génère un fichier JSON.
    """
    # 1. Chargement des termes protégés (Mots Scientifiques)
    try:
        with open(protected_terms_path, 'r', encoding='utf-8') as f:
            # protected_terms.json semble être une simple liste de chaînes
            protected_terms_list = json.load(f)
            # Convertir la liste en un ensemble (set) pour une recherche O(1) rapide
            protected_terms_set = {term.lower() for term in protected_terms_list}
        print(f"✅ Chargement réussi de {len(protected_terms_set)} termes scientifiques/protégés.")
    except FileNotFoundError:
        print(f"❌ Erreur: Le fichier des termes protégés n'a pas été trouvé à {protected_terms_path}")
        return
    except json.JSONDecodeError:
        print(f"❌ Erreur de décodage JSON dans le fichier {protected_terms_path}")
        return

    # 2. Chargement des tokens de concepts
    try:
        with open(tokens_path, 'r', encoding='utf-8') as f:
            # token_concept.json semble être une simple liste de chaînes
            concept_tokens_list = json.load(f)
        print(f"✅ Chargement réussi de {len(concept_tokens_list)} tokens de concepts.")
    except FileNotFoundError:
        print(f"❌ Erreur: Le fichier des tokens n'a pas été trouvé à {tokens_path}")
        return
    except json.JSONDecodeError:
        print(f"❌ Erreur de décodage JSON dans le fichier {tokens_path}")
        return

    # 3. Détection des concepts scientifiques
    scientific_concepts = set()
    
    # Parcourir chaque token et vérifier s'il est dans l'ensemble des termes protégés.
    # On met tout en minuscules pour assurer une correspondance insensible à la casse.
    for token in concept_tokens_list:
        token_lower = token.lower()
        if token_lower in protected_terms_set:
            scientific_concepts.add(token) # Conserver la casse originale du token

    scientific_concepts_list = sorted(list(scientific_concepts))
    print(f"🔍 Détection terminée. {len(scientific_concepts_list)} concepts scientifiques trouvés.")

    # 4. Enregistrement du résultat
    try:
        # Assurez-vous que le répertoire de sortie existe
        output_dir = os.path.dirname(output_path)
        if output_dir: # Ajout d'une vérification pour s'assurer que le chemin n'est pas vide
             os.makedirs(output_dir, exist_ok=True)
            
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(scientific_concepts_list, f, indent=2, ensure_ascii=False)
        print(f"💾 Succès: Les concepts scientifiques ont été enregistrés dans {output_path}")

    except Exception as e:
        print(f"❌ Erreur lors de l'enregistrement du fichier {output_path}: {e}")

# --- Exécution du script ---
if __name__ == "__main__":
    # --- Ligne supprimée ou remplacée : ---
    # L'ancien code causait l'erreur: os.makedirs(os.path.dirname(TOKEN_CONCEPTS_FILE), exist_ok=True)
    # Remplacé par une simple vérification de chemin si nécessaire :
    if os.path.dirname(TOKEN_CONCEPTS_FILE):
        os.makedirs(os.path.dirname(TOKEN_CONCEPTS_FILE), exist_ok=True)
    
    # Le script suppose que vous avez créé les fichiers d'entrée avant de l'exécuter.
    # Exécutez la fonction de détection
    detect_scientific_concepts(TOKEN_CONCEPTS_FILE, PROTECTED_TERMS_FILE, OUTPUT_FILE)

✅ Chargement réussi de 615 termes scientifiques/protégés.
✅ Chargement réussi de 3592 tokens de concepts.
🔍 Détection terminée. 78 concepts scientifiques trouvés.
💾 Succès: Les concepts scientifiques ont été enregistrés dans science_concept.json
